In [165]:
data = pd.read_csv("정리데이터/train_0926.csv")
submission = pd.read_csv("submission.csv")

In [166]:
submission

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가
0,2020-06-01 6:20,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,"59,800"
1,2020-06-01 6:40,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,"59,800"
2,2020-06-01 7:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,"59,800"
3,2020-06-01 7:20,20.0,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,"69,900"
4,2020-06-01 7:40,20.0,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,"69,900"
...,...,...,...,...,...,...,...
2886,2020-07-01 0:20,20.0,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,-
2887,2020-07-01 0:40,20.0,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,-
2888,2020-07-01 1:00,20.0,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,-
2889,2020-07-01 1:20,20.0,100261,200875,아놀드파마 티셔츠레깅스세트,의류,"69,900"


In [2]:
import pandas as pd
import numpy as np

In [11]:
lotte_tv = pd.read_csv("./제출데이터셋/타방송편성표/lotte_tv.csv")
gs_tv = pd.read_csv("./제출데이터셋/타방송편성표/gs_tv.csv")
cj_tv = pd.read_csv("./제출데이터셋/타방송편성표/cj_tv.csv")

## 타 편성표 데이터 전처리
- 타 홈쇼핑의 세일 유무 칼럼 만들기

In [42]:
sale_words = ["세일","할인","한정","특집","특가","최저가","인하","1+1", "SALE"]

def is_hot_deal(x):
    for w in sale_words:
        if w in x:
            return True
        else:
            pass
    return False
        
def hot_preprocessing(data):
    hot_deal_data = data[data["상품명"].apply(is_hot_deal)] # 핫딜인 상품만을 추출
    
    hot_deal_bool = hot_deal_data[["날짜","시"]]
    hot_deal_bool["세일"]=1
    
    return hot_deal_bool

In [43]:
# 각 홈쇼핑 데이터에서 세일하는 날짜, 시 추출
lotte_hot_bool = hot_preprocessing(lotte_tv)
gs_hot_bool = hot_preprocessing(gs_tv)
cj_hot_bool = hot_preprocessing(cj_tv)

# 각 홈쇼핑 데이터 합침
total_hot_bool = pd.concat([cj_hot_bool, gs_hot_bool, lotte_hot_bool], axis=0)

In [44]:
# 각 홈쇼핑 데이터를 날짜와 시 기준으로 count
hot = total_hot_bool.groupby(["날짜","시"]).count().reset_index()

In [222]:
# merge 를 위한 기존 데이터에서 기준 칼럼 생성
submission["시"] = pd.to_datetime(submission["방송일시"]).dt.hour
submission["날짜"] = pd.to_datetime(submission["방송일시"]).dt.date
submission["요일"] = pd.to_datetime(submission["방송일시"]).dt.day

submission["날짜"] = submission["날짜"].astype(str)
submission["시"] = submission["시"].astype(str)

# 기존 데이터와 merge
submission = submission.merge(hot, on=["시","날짜"], how="left")
submission["세일"]=submission["세일"].fillna(0)

(2891, 10)


In [223]:
# 실제날씨, 앱유저, 예보날씨 기존 데이터와 merge
a = pd.read_csv("./제출데이터셋/실제날씨.csv")
b = pd.read_csv("./제출데이터셋/앱유저.csv")
c = pd.read_csv("./제출데이터셋/예보날씨.csv")

submission = submission.merge(a, how="left", on=["날짜"])
submission = submission.merge(b, how="left", on=["날짜"])
submission = submission.merge(c, how="left", on=["날짜"])

# 주말 및 휴일 칼럼 만들기(6월은 공휴일이 하루 -> 토요일과 겹침)
submission['holiday(includeSS)']=0
submission['holiday(includeSS)'][(submission["요일"]==5)|(submission["요일"]==6)]=1

# 노출분 null 값 바로 전 값으로 채우기
submission["노출(분)"] = submission["노출(분)"].fillna(method = "ffill")

# 판매단가 칼럼 null 값 제거 및 float 형으로 변환
submission["판매단가"] = submission["판매단가"].str.replace(",","")
submission["판매단가"] = submission["판매단가"].str.replace("-","").astype(float)

(2891, 14)


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [231]:
submission.to_csv("./제출데이터셋/submission.csv", index=False) # 저장